In [1]:
import numpy as np
jobs = np.array(
    [[[[True,  True, False],
        [True, False,  True],
        [True, False,  True],
        [False, False,  True],
        [False,  True,  True]],

      [[False, False,  True],
        [True,  True,  True],
        [False,  True,  True],
        [False, False, False],
        [False,  True, False]]],


      [[[False, False,  True],
        [False,  True, False],
        [True,  True,  True],
        [False, False,  True],
        [False, False, False]],

      [[False, False,  True],
        [True, False, False],
        [True, False,  True],
        [True,  True, False],
        [True, False, False]]],


      [[[True, False, False],
        [False, False,  True],
        [True, False,  True],
        [True,  True, False],
        [False, False,  True]],

      [[True,  True,  True],
        [True, False,  True],
        [True,  True,  True],
        [False, False,  True],
        [False, False, False]]],


      [[[False, False,  True],
        [True,  True, False],
        [False,  True, False],
        [False,  True,  True],
        [True, False, False]],

      [[False,  True, False],
        [False,  True, False],
        [True,  True, False],
        [False, False, False],
        [True, False,  True]]],


      [[[True, False,  True],
        [False,  True,  True],
        [False,  True,  True],
        [True, False, False],
        [True, False,  True]],

      [[True,  True, False],
        [False,  True,  True],
        [False,  True,  True],
        [True, False,  True],
        [True,  True,  True]]]]
)
from res_mgmt.envs.res_mgmt_env import ResMgmtEnv

pygame 2.1.2 (SDL 2.0.16, Python 3.10.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

env = ResMgmtEnv(
    num_resource_type=2,
    resource_size=3,
    time_size=5,
    num_job_slot=3,
    max_num_job=10**3,
    jobs=np.copy(jobs),
)
obs = env.reset()
print(obs.shape)

action = env.action_space.sample()
print("Action:\n", action)
obs, reward, done, info = env.step(action)
print("Step:\n", obs, reward, done, info)

pygame 2.1.2 (SDL 2.0.16, Python 3.10.2)
Hello from the pygame community. https://www.pygame.org/contribute.html
(121,)
Action:
 0
Step:
 [1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000 1000
 1000 1000    1    1    0    1    0    1    1    0    1    0    0    1
    0    1    1    0    0    1    1    1    1    0    1    1    0    0
    0    0    1    0    0    0    1    0    1    0    1    1    1    0
    0    1    0    0    0    0    0    1    1    0    0    1    0    1
    1    1    0    1    0    0    1    0    0    0    0    1    1    0
    1    1    1    0    0    0    1    1    1    1    1    0    1    1
    1    1    0    0    1    0    0    0    2] -1.0 False {}


In [ ]:
from stable_baselines3.common.env_checker import check_env
env = ResMgmtEnv(
    num_resource_type=2,
    resource_size=3,
    time_size=5,
    num_job_slot=3,
    max_num_job=10**3,
    jobs=np.copy(jobs),
)
check_env(env, warn=True)

In [ ]:
from stable_baselines3 import A2C

env = ResMgmtEnv(
    num_resource_type=2,
    resource_size=3,
    time_size=5,
    num_job_slot=3,
    max_num_job=10**3,
    jobs=np.copy(jobs),
)

model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=10)

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

env = ResMgmtEnv(
    num_resource_type=2,
    resource_size=3,
    time_size=5,
    num_job_slot=3,
    max_num_job=10**3,
    jobs=np.copy(jobs),
)
model = A2C('MlpPolicy', env, verbose=1)

evaluate_policy(model, env, n_eval_episodes=2)


In [2]:
import pygame
env = ResMgmtEnv(
    num_resource_type=2,
    resource_size=3,
    time_size=5,
    num_job_slot=3,
    max_num_job=10**3,
    jobs=np.copy(jobs),
)

observation = env.reset()
t = 0
while True:
    t += 1
    # print(observation)
    action = env.action_space.sample()
    observation, reward, done, info = env.step(action)
    print(t, action, env.res.empty_cells_cluster)
    env.my_render(f"render/{t}.png")
    if done or t > 10:
        print("Episode finished after {} timesteps".format(t+1))
        break
env.close()

1 2 [[2 2 0 2 3]
 [2 2 1 1 2]]
2 1 [[2 0 2 3 3]
 [2 1 1 2 3]]
3 2 [[0 2 3 3 3]
 [1 1 2 3 3]]
4 1 [[2 3 3 3 3]
 [1 2 3 3 3]]
5 1 [[3 3 3 3 3]
 [2 3 3 3 3]]
6 1 [[1 1 1 2 1]
 [1 0 1 3 2]]
7 3 [[1 1 2 1 3]
 [0 1 3 2 3]]
8 0 [[1 2 1 3 3]
 [1 3 2 3 3]]
9 1 [[2 1 3 3 3]
 [3 2 3 3 3]]
10 1 [[1 3 3 3 3]
 [2 3 3 3 3]]
11 3 [[3 3 3 3 3]
 [3 3 3 3 3]]
Episode finished after 12 timesteps


In [ ]:
from IPython.display import Image
Image(filename="output.png") 